# Crisis severity quintile

## Data cleaning and preparation for PCA
### Version 2: 2020 as base year, en utilisant les critical indicators des LSG

In [2]:
#libraries imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_rows',None)

In [3]:
#import the clean data
df1 = pd.read_excel ("master_dataset.xlsx")

In [4]:
#create a copy to avoid reloading the df everytime since it is very heavy
df = df1.copy()

In [5]:
#looking at missing values
df.isna().sum()

0                                                            0
X_uuid                                                      25
population_group                                             0
district                                                     0
governorate                                                  0
gender_head                                               6422
num_hh_member                                               70
weight                                                       0
year                                                         0
received_aid                                                 0
received_aid_cash                                        33182
received_aid_food                                        33182
received_aid_water                                       33182
received_aid_fuel                                        33182
received_aid_shelter                                     33182
received_aid_seasonal_items                            

#### selecting only the needed years

In [6]:
#keeping only 2020-2021
df.drop(df1[df1["year"]==2017].index, inplace=True)
df.drop(df1[df1["year"]==2018].index, inplace=True)
df.drop(df1[df1["year"]==2019].index, inplace=True)

#### Listing the Critical LSG indicators and keeping only those

In [7]:
#keep only the CRITICAL indicators in the LSG
LSGcapacitygap = ["ngo_assistance", "stress", "crisis", "emergency"]
LSGvulenrabilities = ["single_headed", "female_headed"]
LSGeducation =["not_attending_formal"]
LSGFS = ["fcs_category","household_hunger_category"]
LSGprotection = ["child_married", "children_working"]
LSGSNFI = ["critical_shelter"]
LSGwash = ["imptoved_water_source", "insufficient_amount_water"] #missing some variables here (dont exist in our dataset)
LSGall = LSGcapacitygap +LSGvulenrabilities +LSGeducation+LSGFS+LSGprotection+LSGSNFI+LSGwash

In [8]:
#other columns needed later
needed = ["year", "X_uuid", "population_group", "district", "governorate", "weight"]

In [9]:
#all columns to keep
tokeep = LSGall + needed

In [10]:
df = df[df.columns[df.columns.isin(tokeep)]]

In [11]:
df.head()

,X_uuid,population_group,district,governorate,weight,year,critical_shelter,not_attending_formal,fcs_category,household_hunger_category,stress,crisis,emergency,female_headed,single_headed,children_working,ngo_assistance,child_married,imptoved_water_source,insufficient_amount_water
0,14c9f242-8eb3-4f00-b724-60e51d9d244e,idp_out_camp,al.hilla,babil,0.032164,2021,0.0,0.0,Poor,Little to no hunger in the household (0-1),1.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0,0.0
1,10ec8c64-4f0b-45c9-a44e-7a50619757b4,returnee,al.muqdadiya,diyala,0.570485,2021,0.0,NaN,Acceptable,Little to no hunger in the household (0-1),1.0,1.0,0.0,0,0.0,NaN,0.0,NaN,0,0.0
2,8929207e-4cff-4089-b626-69bd00039676,returnee,al.muqdadiya,diyala,0.570485,2021,0.0,NaN,Acceptable,Little to no hunger in the household (0-1),1.0,1.0,0.0,1,1.0,NaN,0.0,NaN,0,1.0
3,b5ef8e6c-0ca7-46bb-a901-67a957e3aa0a,returnee,al.muqdadiya,diyala,0.570485,2021,0.0,NaN,Acceptable,Little to no hunger in the household (0-1),1.0,0.0,0.0,0,0.0,NaN,0.0,NaN,0,0.0
4,2be976e3-795d-4a09-8a56-e4f0a67811ad,returnee,al.muqdadiya,diyala,0.570485,2021,0.0,NaN,Acceptable,Little to no hunger in the household (0-1),0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,1.0


## handling missing values

In [12]:
pd.set_option('display.max_rows',None)
df.isna().sum()

X_uuid                          0
population_group                0
district                        0
governorate                     0
weight                          0
year                            0
critical_shelter                0
not_attending_formal         7450
fcs_category                    0
household_hunger_category       0
stress                          0
crisis                          0
emergency                       0
female_headed                   0
single_headed                   0
children_working             4269
ngo_assistance                  0
child_married                4274
imptoved_water_source           0
insufficient_amount_water       0
dtype: int64

##### It seems that when children_working or not_attening_formal is missing, it is because there is no children in the HH. I still have doubts because there is not the same number of missing variables for both. This is a caveat to keep in mind, especially now that these are 2 out of a few variables only.

In [13]:
df["children_working"].replace(np.nan,0, inplace=True) #the missing values are because these HH have no children

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [14]:
df["not_attending_formal"].replace(np.nan,0, inplace=True) #same, missing because have no children (?)

In [15]:
df["child_married"].replace(np.nan,0, inplace=True)

### One hot encoding fcs_category and household_hunder_category

In [16]:
dfa = pd.get_dummies(df["fcs_category"], prefix='fcs')
df = pd.concat([df, dfa], axis=1)
df.drop(["fcs_category"], axis=1, inplace=True)

In [17]:
dfa = pd.get_dummies(df["household_hunger_category"], prefix='hhs')
df = pd.concat([df, dfa], axis=1)
df.drop(["household_hunger_category"], axis=1, inplace=True)

In [18]:
df.head()

,X_uuid,population_group,district,governorate,weight,year,critical_shelter,not_attending_formal,stress,crisis,...,ngo_assistance,child_married,imptoved_water_source,insufficient_amount_water,fcs_Acceptable,fcs_Borderline,fcs_Poor,hhs_Little to no hunger in the household (0-1),hhs_Moderate hunger in the household (2-3),hhs_Severe hunger in the household (4-6)
0,14c9f242-8eb3-4f00-b724-60e51d9d244e,idp_out_camp,al.hilla,babil,0.032164,2021,0.0,0.0,1.0,0.0,...,0.0,0.0,0,0.0,0,0,1,1,0,0
1,10ec8c64-4f0b-45c9-a44e-7a50619757b4,returnee,al.muqdadiya,diyala,0.570485,2021,0.0,0.0,1.0,1.0,...,0.0,0.0,0,0.0,1,0,0,1,0,0
2,8929207e-4cff-4089-b626-69bd00039676,returnee,al.muqdadiya,diyala,0.570485,2021,0.0,0.0,1.0,1.0,...,0.0,0.0,0,1.0,1,0,0,1,0,0
3,b5ef8e6c-0ca7-46bb-a901-67a957e3aa0a,returnee,al.muqdadiya,diyala,0.570485,2021,0.0,0.0,1.0,0.0,...,0.0,0.0,0,0.0,1,0,0,1,0,0
4,2be976e3-795d-4a09-8a56-e4f0a67811ad,returnee,al.muqdadiya,diyala,0.570485,2021,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1.0,1,0,0,1,0,0


### saving final reduced dataset

In [19]:
df.to_excel("final_clean_v2.xlsx", index=False)

### Checking how many rows are identical

Problem: because I only have a few variables in this dataset, and that they are all dummies, there are many rows taht are perfectly identical. That is: There are 3140 rows that are exactly identical one to another, a second group of 2780 identical, then 852, 777, ... 

This leads to a problem in the PCA whereby there are many observations that obtain the exact same score. It also leads the Principal components to not be continuous, but almost categorical

In [20]:
df.columns

Index(['X_uuid', 'population_group', 'district', 'governorate', 'weight',
       'year', 'critical_shelter', 'not_attending_formal', 'stress', 'crisis',
       'emergency', 'female_headed', 'single_headed', 'children_working',
       'ngo_assistance', 'child_married', 'imptoved_water_source',
       'insufficient_amount_water', 'fcs_Acceptable', 'fcs_Borderline',
       'fcs_Poor', 'hhs_Little to no hunger in the household (0-1)',
       'hhs_Moderate hunger in the household (2-3)',
       'hhs_Severe hunger in the household (4-6)'],
      dtype='object')

In [21]:
df["checkunique"] = df["critical_shelter"].astype(str) + df["not_attending_formal"].astype(str)

In [22]:
df["checkunique"] = df["critical_shelter"].astype(str) + df["not_attending_formal"].astype(str) + df["stress"].astype(str) + df["crisis"].astype(str) + df["emergency"].astype(str) + df["female_headed"].astype(str) + df["single_headed"].astype(str) + df["children_working"].astype(str) + df["ngo_assistance"].astype(str) + df["child_married"].astype(str) + df["imptoved_water_source"].astype(str) + df["insufficient_amount_water"].astype(str) + df["fcs_Acceptable"].astype(str) + df["fcs_Borderline"].astype(str) + df["fcs_Poor"].astype(str) + df["hhs_Severe hunger in the household (4-6)"].astype(str) + df["hhs_Little to no hunger in the household (0-1)"].astype(str) + df["hhs_Moderate hunger in the household (2-3)"].astype(str)

In [23]:
dfoccurences = pd.DataFrame(df["checkunique"].value_counts())

In [24]:
dfoccurences = dfoccurences.rename_axis('unique').reset_index()

In [25]:
#the one case that happens 3140 times in the dataset
df[df["checkunique"]==dfoccurences.iloc[0,0]].iloc[0,]
#not enough water, no hunger, fcs acceptable,  stress level (everything else 0)

X_uuid                                              71bf56b7-2797-4929-a486-7f475a3b8bed
population_group                                                            idp_out_camp
district                                                                      chamchamal
governorate                                                              al.sulaymaniyah
weight                                                                          0.108896
year                                                                                2021
critical_shelter                                                                     0.0
not_attending_formal                                                                 0.0
stress                                                                               1.0
crisis                                                                               0.0
emergency                                                                            0.0
female_headed        

In [26]:
#the case that is present 2780 times in the dataset
df[df["checkunique"]==dfoccurences.iloc[1,0]].iloc[0,]
#not hunger, not enough water, fcs acceptable, everything else 0

X_uuid                                              2be976e3-795d-4a09-8a56-e4f0a67811ad
population_group                                                                returnee
district                                                                    al.muqdadiya
governorate                                                                       diyala
weight                                                                          0.570485
year                                                                                2021
critical_shelter                                                                     0.0
not_attending_formal                                                                 0.0
stress                                                                               0.0
crisis                                                                               0.0
emergency                                                                            0.0
female_headed        

In [27]:
#present 852 times in the dataset
df[df["checkunique"]==dfoccurences.iloc[3,0]].iloc[0,]
#no hunger, fcs acceptable, stress level, critical shelter

X_uuid                                              c1d4c9a8-74da-4968-b7d5-7243bcbc2ecd
population_group                                                            idp_out_camp
district                                                                          sinjar
governorate                                                                       ninewa
weight                                                                            0.3789
year                                                                                2021
critical_shelter                                                                     1.0
not_attending_formal                                                                 0.0
stress                                                                               1.0
crisis                                                                               0.0
emergency                                                                            0.0
female_headed        